In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Tue Aug  6 07:38:18 PDT 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
roadrunner

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                16
On-line CPU(s) list:   0-15
Thread(s) per core:    2
Core(s) per socket:    4
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 26
Model name:            Intel(R) Xeon(R) CPU           E5520  @ 2.27GHz
Stepping:              5
CPU MHz:               2394.000
CPU max MHz:           2394.0000
CPU min MHz:           1596.0000
BogoMIPS:              4521.81
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              8192K
NUMA node0 CPU(s):     0-15
Flags:                 fpu vme de pse tsc msr pae m

No LSB modules are available.


In [2]:
%env wd=/home/sam/analyses/20190805_Pgenerosa_v070_repeatmasker_all
%env fasta_url=https://owl.fish.washington.edu/halfshell/genomic-databank/Pgenerosa_v070.fa
%env fasta=Pgenerosa_v070.fa
%env repeat_masker=/home/shared/RepeatMasker-4.0.7/RepeatMasker
%env cpus=16
# Checksum taken from https://github.com/RobertsLab/resources/wiki/Genomic-Resources
%env checksum=de0958fec4b9f8845babf3717ce7168c

env: wd=/home/sam/analyses/20190805_Pgenerosa_v070_repeatmasker_all
env: fasta_url=https://owl.fish.washington.edu/halfshell/genomic-databank/Pgenerosa_v070.fa
env: fasta=Pgenerosa_v070.fa
env: repeat_masker=/home/shared/RepeatMasker-4.0.7/RepeatMasker
env: cpus=16
env: checksum=de0958fec4b9f8845babf3717ce7168c


#### Create necessary directories

In [3]:
%%bash
mkdir --parents ${wd}

#### Download _Crassostrea gigas genome FastA file

Info on FastA file is here: [https://github.com/RobertsLab/resources/wiki/Genomic-Resources#genome](https://github.com/RobertsLab/resources/wiki/Genomic-Resources#genome)

In [4]:
%%bash
cd ${wd}

rsync \
--archive \
--verbose \
--progress \
owl:/volume1/web/halfshell/genomic-databank/${fasta} .

echo ""
echo ""
echo "----------------------------------------------------------"

ls -lh

receiving incremental file list
Pgenerosa_v070.fa
  2,247,117,885 100%   17.87MB/s    0:01:59 (xfr#1, to-chk=0/1)

sent 30 bytes  received 2,247,392,295 bytes  18,497,056.17 bytes/sec
total size is 2,247,117,885  speedup is 1.00


----------------------------------------------------------
total 2.1G
-rw-r--r-- 1 sam users 2.1G Feb 11 12:13 Pgenerosa_v070.fa


#### If need to download via wget, change cell below to code, instead of markdown

#### Also, change cell with md5 checksum comparisons to code, instead of markdown

%%bash
time \
wget ${fasta_url} \
--quiet \
--directory-prefix=${wd}

ls -lh ${wd}

#### Verify MD5 checksum

Original MD5 checksum taken from GitHub Genomic Resource linked above.

Use ```md5sum``` to generate checksum from downloaded FastA file and ```awk``` to print the first field (i.e. the checksum value). This is saved to the variable: ```dl_md5```

Then, check for differences between the two variables. 

No output confirms no difference.

%%bash
md5=${checksum}
dl_md5=$(md5sum ${wd}/${fasta} | awk '{ print $1 }')
diff <(echo "$md5") <(echo "$dl_md5")

### Trim FastA header length to avoid RepeatMasker error of length >50 characters

In [5]:
%%bash
cd ${wd}
echo "Original FastA header:"
head -n 1 "${fasta}"

echo ""

cut -f 1-2 -d "_" "${fasta}" > Pgenerosa_v070.fa.tmp
mv Pgenerosa_v070.fa.tmp "${fasta}"

echo "---------------------------------"

echo "Updated FastA header:"
head -n 1 "${fasta}"

ls -ltrh

Original FastA header:

---------------------------------
Updated FastA header:
>PGA_scaffold1
total 2.1G
-rw-rw-r-- 1 sam sam 2.1G Aug  6 07:48 Pgenerosa_v070.fa


bash: line 3: head-n: command not found


#### Run RepeatMasker with _all_ species setting and following options:

```-species "all"``` : Sets species to all

```-par ${cpus}``` : Use n CPU threads

```-gff``` : Create GFF output file (in addition to default files)

```-excln``` : Adjusts output table calculations to exclude sequence runs of >=25Ns. Useful for draft genome assemblies.

```-1>``` : Send stdout to file instead of printing to notebook.

```-2>``` : Send stderr to file instead of printing to notebook.


In [6]:
%%bash

cd ${wd}
time \
${repeat_masker} \
${fasta} \
-species "all" \
-par ${cpus} \
-gff \
-excln \
1> stdout.out \
2> stderr.err

sed '/^Subject:/ s/ / repeatmasker_pgenv070_all JOB COMPLETE/' ~/.default-subject.mail | msmtp "$EMAIL"


real	5729m38.010s
user	88723m33.224s
sys	786m15.300s


In [7]:
%%bash
ls -lh ${wd}

total 5.1G
-rw-rw-r-- 1 sam sam 2.1G Aug  6 07:48 Pgenerosa_v070.fa
-rw-rw-r-- 1 sam sam 438M Aug 10 07:19 Pgenerosa_v070.fa.cat.gz
-rw-rw-r-- 1 sam sam 2.2G Aug 10 07:18 Pgenerosa_v070.fa.masked
-rw-rw-r-- 1 sam sam 254M Aug 10 07:19 Pgenerosa_v070.fa.out
-rw-rw-r-- 1 sam sam 157M Aug 10 07:18 Pgenerosa_v070.fa.out.gff
-rw-rw-r-- 1 sam sam 2.4K Aug 10 07:18 Pgenerosa_v070.fa.tbl
-rw-rw-r-- 1 sam sam    0 Aug  6 07:49 stderr.err
-rw-rw-r-- 1 sam sam 6.8M Aug 10 07:18 stdout.out


#### SUMMARY TABLE

In [8]:
%%bash
cat ${wd}/${fasta}.tbl

file name: Pgenerosa_v070.fa        
sequences:        313649
total length: 2205688688 bp  (2005531528 bp excl N/X-runs)
GC level:         33.92 %
bases masked:  175175579 bp ( 8.73 %)
               number of      length   percentage
               elements*    occupied  of sequence
--------------------------------------------------
Retroelements       565711     87788537 bp    4.38 %
   SINEs:           332333     39506023 bp    1.97 %
   Penelope           6883       788411 bp    0.04 %
   LINEs:           142649     32744907 bp    1.63 %
    CRE/SLACS         1237       100944 bp    0.01 %
     L2/CR1/Rex      40317      7764197 bp    0.39 %
     R1/LOA/Jockey   10137      2942539 bp    0.15 %
     R2/R4/NeSL       3825       551996 bp    0.03 %
     RTE/Bov-B       26939      6768723 bp    0.34 %
     L1/CIN4         21435      4046589 bp    0.20 %
   LTR elements:     90729     15537607 bp    0.77 %
     BEL/Pao          6594       918331 bp    0.05 %
     Ty1/Copia       16409  

#### Delete FastA (not needed) and `rsync` to my folder on Gannet

In [11]:
%%bash
cd /home/sam/analyses/

rsync \
--archive \
--verbose \
--progress \
--relative \
./20190805_Pgenerosa_v070_repeatmasker_all \
gannet:/volume2/web/Atumefaciens

sending incremental file list

sent 288 bytes  received 16 bytes  608.00 bytes/sec
total size is 2,258,210,520  speedup is 7,428,324.08
